<a href="https://colab.research.google.com/github/FeelOneE/SQuAD/blob/main/SQuAD_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[참고 자료](https://colab.research.google.com/github/huggingface/datasets/blob/master/notebooks/Overview.ipynb#scrollTo=zNp6kK7OvSUg)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/스터디/기계이해/squad_1.1v/
% ls -al

/content/drive/MyDrive/스터디/기계이해/squad_1.1v
total 34440
-rw------- 1 root root  4854279 Nov 17  2019  dev-v1.1.json
-rw------- 1 root root   122790 Jan 21 02:57 'SQuAD 1.1.ipynb'
-rw------- 1 root root 30288272 Nov 17  2019  train-v1.1.json


# 1. 데이터 로드

In [3]:
import pandas as pd

In [35]:
train = pd.read_json('train-v1.1.json')
valid = pd.read_json('train-v1.1.json')

In [36]:
train.shape, valid.shape

((442, 2), (442, 2))

In [37]:
train.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [41]:
train.iloc[1,0]['paragraphs']

[{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
    'id': '56be85543aeaaa14008c9063',
    'question': 'When did Beyonce start becoming popular?'},
   {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
    'id': '56be85543aeaaa14008c9065',
  

# 2.훈련 데이터 형식 맞추기

In [43]:
# 제일 깊이(depth)가 낮은 데이터를 기준으로 row를 생성함
# answers, text
# https://github.com/aswalin/SQuAD/blob/master/create_emb.ipynb

contexts = []
questions = []
answers_text = []
answers_start = []

for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [44]:
df.head(5)

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [46]:
df.shape

(87599, 4)

In [ ]:
!pip install transformers

In [47]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained("bert-base-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
!pip install datasets

     |████████████████████████████████| 306 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.9 MB/s 
     |████████████████████████████████| 133 kB 48.5 MB/s 
     |████████████████████████████████| 243 kB 49.4 MB/s 
     |████████████████████████████████| 144 kB 49.8 MB/s 
     |████████████████████████████████| 271 kB 48.3 MB/s 
     |████████████████████████████████| 160 kB 49.3 MB/s 


---- dataset 라이브러리로 불러오기 ----


In [51]:
from datasets import list_datasets, list_metrics, load_dataset, load_metric
from pprint import pprint

In [52]:
datasets = list_datasets()
metrics = list_metrics()

squad_dataset = list_datasets(with_details=True)[datasets.index('squad')]
pprint(squad_dataset.__dict__)  # It's a simple python dataclass

{'author': None,
 'cardData': {'annotations_creators': ['crowdsourced'],
              'language_creators': ['crowdsourced', 'found'],
              'languages': ['en'],
              'licenses': ['cc-by-4.0'],
              'multilinguality': ['monolingual'],
              'paperswithcode_id': 'squad',
              'pretty_name': 'SQuAD',
              'size_categories': ['10K<n<100K'],
              'source_datasets': ['extended|wikipedia'],
              'task_categories': ['question-answering'],
              'task_ids': ['extractive-qa']},
 'citation': '@article{2016arXiv160605250R,\n'
             '       author = {{Rajpurkar}, Pranav and {Zhang}, Jian and '
             '{Lopyrev},\n'
             '                 Konstantin and {Liang}, Percy},\n'
             '        title = "{SQuAD: 100,000+ Questions for Machine '
             'Comprehension of Text}",\n'
             '      journal = {arXiv e-prints},\n'
             '         year = 2016,\n'
             '          eid 

In [53]:
dataset = load_dataset('squad', split='validation[:10%]')

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [54]:
pprint(dataset.info.__dict__)

{'builder_name': 'squad',
 'citation': '@article{2016arXiv160605250R,\n'
             '       author = {{Rajpurkar}, Pranav and {Zhang}, Jian and '
             '{Lopyrev},\n'
             '                 Konstantin and {Liang}, Percy},\n'
             '        title = "{SQuAD: 100,000+ Questions for Machine '
             'Comprehension of Text}",\n'
             '      journal = {arXiv e-prints},\n'
             '         year = 2016,\n'
             '          eid = {arXiv:1606.05250},\n'
             '        pages = {arXiv:1606.05250},\n'
             'archivePrefix = {arXiv},\n'
             '       eprint = {1606.05250},\n'
             '}\n',
 'config_name': 'plain_text',
 'dataset_size': 89819400,
 'description': 'Stanford Question Answering Dataset (SQuAD) is a reading '
                'comprehension dataset, consisting of questions posed by '
                'crowdworkers on a set of Wikipedia articles, where the answer '
                'to every question is a segment of

In [55]:
dataset[1]

{'answers': {'answer_start': [249, 249, 249],
  'text': ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers']},
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'id': '56be4db0acb8001400a502ed',
 'question': 'Which NFL team represented the NFC at

In [56]:
dataset[2]

{'answers': {'answer_start': [403, 355, 355],
  'text': ['Santa Clara, California',
   "Levi's Stadium",
   "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]},
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'id': '56be4db0acb800

# 토크나이저 로드

In [64]:
from transformers import BertTokenizerFast, logging as transformers_logging

#  BertTokenizer는 char_to_token(문서의 인덱스를 토큰의 인덱스로 변환) 사용 불가
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [65]:
# dataset의 context를 토크나이저에 넣어 반환된 값을 datset에 추가 (input_ids, attention_mask, token_type_ids) 추가됨
encoded_dataset = dataset.map(lambda example: tokenizer(example['context']), batched=True)

print("encoded_dataset[0]")
pprint(encoded_dataset[0], compact=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

encoded_dataset[0]
{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                    1],
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Co

In [67]:
pprint(dataset.column_names)

['id', 'title', 'context', 'question', 'answers']


In [68]:
# 문서 기준 시작 위치 인덱스를 토큰 기준으로 변환

def convert_to_features(batch):
    # Tokenize contexts and questions (as pairs of inputs)
    input_pairs = list(zip())

    # 토크나이저에 두개의 문장을 넣으면 token_type_ids에서 구분할 수 있음
    encodings = tokenizer(batch['context'], batch['question'], truncation=True)

    # Compute start and end tokens for labels
    start_positions, end_positions = [], []
    # 배치로 여러 문서가 들어오니 answer는 answer_start, text를 가지고 있는 데이터
    for i, answer in enumerate(batch['answers']):
        first_char = answer['answer_start'][0]
        last_char = first_char + len(answer['text'][0]) - 1
        start_positions.append(encodings.char_to_token(i, first_char))
        end_positions.append(encodings.char_to_token(i, last_char))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

encoded_dataset = dataset.map(convert_to_features, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [82]:
print("column_names", encoded_dataset.column_names)
print("start_positions", encoded_dataset[:5]['start_positions'])

column_names ['answers', 'attention_mask', 'context', 'end_positions', 'id', 'input_ids', 'question', 'start_positions', 'title', 'token_type_ids']
start_positions tf.Tensor([34 45 80 34 98], shape=(5,), dtype=int64)


In [71]:
columns_to_return = ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']

# Uncomment whichever one is appropriate for you
# encoded_dataset.set_format(type='torch', columns=columns_to_return)
encoded_dataset.set_format(type='tensorflow', columns=columns_to_return)

# Our dataset indexing output is now ready for being used in a pytorch dataloader
pprint(encoded_dataset[1], compact=True)

{'attention_mask': <tf.Tensor: shape=(172,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>,
 'end_positions': <tf.Tensor: shape=(), dtype=int64, numpy=46>,
 'input_ids': <tf.Tensor: shape=(172,), dtype=int64, numpy=
array([  101,  3198,  5308,  1851,  1108,  1126,  1237,  1709,  1342,
        1106,  4959,  1103,  3628,  1104,  1103,  1305,  2289,  1453,
         113,  4279,   114,  1111,  1103,  1410,  1265,   119,  1109,
        1237,  2289,  3047

In [83]:
print(encoded_dataset.column_names)

['answers', 'attention_mask', 'context', 'end_positions', 'id', 'input_ids', 'question', 'start_positions', 'title', 'token_type_ids']


In [ ]:
!pip install transformers

In [89]:
# tf dataset로 변환하기
columns = ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']

# Let's do dynamic batching (pad on the fly with our own collate_fn)
def collate_fn(examples):
    return tokenizer.pad(examples, return_tensors='np')

# to_tf_dataset() returns a tf.data.Dataset that we can pass straight to model.fit().
# Note the use of dummy_labels - this isn't always necessary, but can stop TF getting confused when you don't pass any labels.
encoded_tf_dataset = encoded_dataset.to_tf_dataset(
    columns=columns,
    collate_fn=collate_fn,
    batch_size=8,
    shuffle=True,
    dummy_labels=True
)

In [92]:
# 데이터 셋 확인하기
cnt = 0
for i in encoded_tf_dataset:
  pprint(i)
  cnt += 1
  if cnt > 0:
    break

({'attention_mask': <tf.Tensor: shape=(8, 208), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>,
  'end_positions': <tf.Tensor: shape=(8,), dtype=int64, numpy=array([ 62, 154, 120,  26, 102,  17, 111,  47])>,
  'input_ids': <tf.Tensor: shape=(8, 208), dtype=int64, numpy=
array([[  101, 19289, 13489, ...,     0,     0,     0],
       [  101,  7068,  1261, ...,     0,     0,     0],
       [  101, 16752, 14840, ...,  5308,   136,   102],
       ...,
       [  101,  1109, 13598, ...,     0,     0,     0],
       [  101,  2409,  1242, ...,     0,     0,     0],
       [  101,  1109,  3198, ...,     0,     0,     0]])>,
  'start_positions': <tf.Tensor: shape=(8,), dtype=int64, numpy=array([ 62, 152, 117,  24, 102,  16, 109,  45])>,
  'token_type_ids': <tf.Tensor: shape=(8, 208), dtype=int64, numpy=
array([[0, 0, 0, 

In [58]:
%tensorflow_version 2.x
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, TFAutoModelForQuestionAnswering
from transformers import EarlyStoppingCallback



In [85]:
# bert-base-uncased 는 vocab 를 의미하며 
# DitilBERT는 BERT 베이스로 훈련된 작고 빠르고 저렴하며 가벼운 Transformer 모델입니다. 
# bert-base-uncased 보다 매개변수가 40% 적고 , GLUE 언어 이해 벤치마크에서 측정한 BERT 성능의 95% 이상을 유지하면서 60% 더 빠르게 실행됩니다.
model = TFAutoModelForQuestionAnswering.from_pretrained('distilbert-base-cased')
# No loss argument!
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5))

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss s

In [86]:
model.summary()

Model: "tf_distil_bert_for_question_answering_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 65190912  
 nLayer)                                                         
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 65,192,450
Trainable params: 65,192,450
Non-trainable params: 0
_________________________________________________________________


In [93]:
model.fit(encoded_tf_dataset, epochs=1, steps_per_epoch=3)

3/3 [==============================] - 67s 12s/step - loss: 5.6918


In [ ]:
model.save("squad_1.1")

# 모델 평가

In [ ]:
!pip install sacrebleu

In [101]:
encoded_dataset[0]

{'attention_mask': <tf.Tensor: shape=(172,), dtype=int64, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>,
 'end_positions': <tf.Tensor: shape=(), dtype=int64, numpy=35>,
 'input_ids': <tf.Tensor: shape=(172,), dtype=int64, numpy=
 array([  101,  3198,  5308,  1851,  1108,  1126,  1237,  1709,  1342,
         1106,  4959,  1103,  3628,  1104,  1103,  1305,  2289,  1453,
          113,  4279,   114,  1111,  1103,  1410,  1265,   119,  1109,
         1237, 

In [103]:
predictions = model(encoded_dataset[0])
predictions

TFQuestionAnsweringModelOutput([('loss',
                                 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.1802263], dtype=float32)>),
                                ('start_logits',
                                 <tf.Tensor: shape=(1, 172), dtype=float32, numpy=
                                 array([[ 0.19872543,  0.02918437,  0.14541127,  0.44182724,  0.18949255,
                                          0.13200015,  0.18708932,  0.38661408,  0.29204798,  0.28428876,
                                          0.33051553,  0.42072707,  0.42678642,  0.5378559 ,  0.5096601 ,
                                          0.16690958,  0.26678658,  0.24918932,  0.29099128,  0.21606743,
                                          0.32506415,  0.29775727,  0.3457694 , -0.04376172,  0.3077802 ,
                                          0.12937196,  0.03433505,  0.11233555,  0.23849778,  0.12790945,
                                          0.2156214 ,  0.0036955 ,  0.311900

In [102]:
from datasets import load_metric
sacrebleu_metric = load_metric('sacrebleu')

# If you only have a single iteration, you can easily compute the score like this
predictions = model(encoded_dataset[0])
score = sacrebleu_metric.compute(predictions, references)

# If you have a loop, you can "add" your predictions and references at each iteration instead of having to save them yourself (the metric object store them efficiently for you). For TensorFlow models, you should be looping over your `tf.data.Dataset`, of course, not your `dataloader`!
for batch in dataloader:
    model_input, targets = batch
    predictions = model(model_inputs)
    sacrebleu_metric.add_batch(predictions, targets)
score = sacrebleu_metric.compute()  # Compute the score from all the stored predictions/references

NameError: ignored

In [ ]:
score